In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH14-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH14-Retriever


# VectorStoreRetriever

1. 개요
    - VectorStore에서 데이터 검색 시 사용하는 검색 인터페이스   
    - 주로 임베딩 벡터 기반으로 사용자 쿼리와 유사한 데이터를 검색하는 역할 

2. 주요 특징 
    - 유사도 기반 검색:
        - 벡터 DB에 저장된 벡터 기반으로 입력 쿼리와 가장 유사한 벡터 검색 
        - 유사도 검색 결과에 메타데이터 필터링 적용하여 더욱 정교한 검색 가능 
    - 검색 결과:
        - 검색된 벡터와 함께 연관 정보(문서 내용, 메타데이터)를 함께 반환 
        - 특히 반환된 메타데이터는 후속 작업에 활용에 편리  


[Reference] https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html

# 파일 로딩 ~ 벡터DB 구성 

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [4]:
# 문서 파일 경로 정의 
file_path = './data/appendix-keywords.txt'

# 로더 정의 
loader = TextLoader(file_path)

# 문서 로딩 
documents = loader.load()

# 분할기 정의 
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 문서 분할 
split_docs = text_splitter.split_documents(documents)

# 임베딩 정의
embeddings = OpenAIEmbeddings()

# Vector DB 정의 
vector_store = FAISS.from_documents(split_docs, embeddings)

# 검색기 (as_retriever)
## Vanilla

In [5]:
# 검색기 변환 
retriever = vector_store.as_retriever()

In [6]:
# 문서 검색 
docs = retriever.invoke('임베딩에 대해 설명해 주세요')

# 상세 결과 출력
for doc in docs:
    print(doc.page_content)
    print('---' * 10)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
------------------------------
정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
------------------------------
정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
------------------------------
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
------------------------------


## MMR

In [7]:
# MMR 기반 검색기 
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k':2,             # 최종적으로 반환할 문서 개수
        'fetch_k':10,      # 후보로 가져올 문서 개수
        'lambda_mult':0.5, # default 0.5, 0: 유사도 점수만 고려, 1: 다양성만 고려
    }
)

In [8]:
# 문서 검색 
docs = retriever.invoke('임베딩에 대해 설명해 주세요')

# 상세 결과 출력
for doc in docs:
    print(doc.page_content)
    print('---' * 10)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
------------------------------
정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이지 랭크 알고리즘을 사용하여 검색 결과의 순위를 정합니다.
연관키워드: 검색 엔진 최적화, 웹 분석, 링크 분석

데이터 마이닝
------------------------------


## 유사도 기반 

In [9]:
# 유사도 & 임계치 기반 검색기
retriever = vector_store.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={
        'score_threshold':0.7,
    }
)

In [10]:
# 문서 검색 
docs = retriever.invoke('임베딩에 대해 설명해 주세요')

# 상세 결과 출력
for doc in docs:
    print(doc.page_content)
    print('---' * 10)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
------------------------------
정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
------------------------------
정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
------------------------------
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
------------------------------


## Configurable

- `ConfigurableField`: 검색 설정을 동적으로 조정하기 위해 사용
- 검색 매개변수의 고유 식별자, 이름, 설명 설정
- 검색 설정은 검색 쿼리와 함께 전달되며, 검색 쿼리에 따라 동적으로 조정

In [11]:
from langchain_core.runnables import ConfigurableField

In [12]:
# 검색기 정의 
# configurable_fields : retriever 설정 필드 정의 (동적 구성)
# ConfigurableField : 특정 설정 필드에 대한 메타데이터 정의 

retriever = vector_store.as_retriever(
    search_kwargs={"k":1}
).configurable_fields(
    # 검색 타입 설정
    search_type=ConfigurableField(
        id='search_type',
        name='Search Type',
        description='The search type to use',
    ),
    # 검색 매개변수 설정 
    search_kwargs=ConfigurableField(
        id='search_kwargs',
        name='Search Kwargs',
        description='The search kwargs to use',
    ),
)

In [13]:
# 검색 설정 정의 
config = {
    'configurable': { # 검색 설정 정의
        'search_kwargs': { # 검색 매개변수
            'k':3,
        },
    },
}

In [14]:
# 문서 검색 
docs = retriever.invoke('임베딩에 대해 설명해 주세요', config=config)

# 상세 결과 출력
for doc in docs:
    print(doc.page_content)
    print('---' * 10)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
------------------------------
정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
------------------------------
정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
------------------------------


In [15]:
# similarity_score_threshold 로 동적 설정한 경우...
config = {
    'configurable': { # 검색 설정 정의
        'search_type':'similarity_score_threshold',
        'search_kwargs': { # 검색 매개변수
            'score_threshold':0.7, 
        },
    },
}

In [16]:
# 문서 검색 
docs = retriever.invoke('임베딩에 대해 설명해 주세요', config=config)

# 상세 결과 출력
for doc in docs:
    print(doc.page_content)
    print('---' * 10)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
------------------------------
정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
------------------------------
정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
------------------------------
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
------------------------------


In [17]:
# MMR 동적 설정한 경우...
config = {
    'configurable': { # 검색 설정 정의
        'search_type':'mmr',
        'search_kwargs': { # 검색 매개변수
            'k': 2,
            'fetch_k': 10,
            'lambda_mult': 0.6, 
        },
    },
}

In [18]:
# 문서 검색 
docs = retriever.invoke('임베딩에 대해 설명해 주세요', config=config)

# 상세 결과 출력
for doc in docs:
    print(doc.page_content)
    print('---' * 10)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
------------------------------
정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
------------------------------


-----
** End of Documents **